
# Content Based Filtering




### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Content-Based Filtering</a></li>
    </ol>
</div>
<br>


<a id="ref1"></a>

# Acquiring the Data


<a id="ref2"></a>

# Preprocessing


In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies/ml-latest/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('Content-Based-movies/ml-latest/ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\('
C:\Users\Arya\AppData\Local\Temp\ipykernel_24680\6725391.py:3: SyntaxWarning: invalid escape sequence '\('
  movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
C:\Users\Arya\AppData\Local\Temp\ipykernel_24680\6725391.py:5: SyntaxWarning: invalid escape sequence '\d'
  movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
C:\Users\Arya\AppData\Local\Temp\ipykernel_24680\6725391.py:7: SyntaxWarning: invalid escape sequence '\('
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [4]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (1995),[Comedy],1995


In [5]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.


In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [7]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp',axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>

# Content-Based recommendation system


Let's now examine how to create **Content-Based** or **Item-Item recommendation systems**.  This method tries to determine a users favorite aspects of an item, and suggests items with similar aspects. In this case, we will try to approximate the users favorite genres from the movie inputs and how they rated them.

Let's begin by creating an input user to recommend movies to:

Notice: to add more movies into the input user, simply add more elements to the **userInput**. Add more! Just be sure to enter with capital letters and additionally if a movie starts with "The", for example the "The Matrix", then enter it in like this: 'Matrix, The'.


In [8]:
userInput = [
            {'title':'Avengers: Age of Ultron', 'rating':5},
            {'title':'Fantastic Four', 'rating':4.5},
            {'title':'Spider-Man 3', 'rating':4},
            {'title':"Amazing Spider-Man, The", 'rating':4.5},
            {'title':'Thor: The Dark World (2013)', 'rating':4.5},
            {'title':'Captain America: The First Avenger', 'rating':5},
            {'title':'Captain America: The Winter Soldier', 'rating':4.75}   
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Avengers: Age of Ultron,5.00
1,Fantastic Four,4.50
2,Spider-Man 3,4.00
3,"Amazing Spider-Man, The",4.50
4,Thor: The Dark World (2013),4.50
5,Captain America: The First Avenger,5.00
6,Captain America: The Winter Soldier,4.75


In [9]:
import re

# Build normalized titles (remove trailing year and collapse spaces)
movies_df["norm_title"] = (
    movies_df["title"].astype(str).str.strip()
    .str.replace(r"\s\(\d{4}\)$", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)
)
inputMovies["norm_title"] = (
    inputMovies["title"].astype(str).str.strip()
    .str.replace(r"\s\(\d{4}\)$", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)
)

# Filter by normalized titles and merge to get movieId with ratings
inputId = movies_df[movies_df["norm_title"].isin(inputMovies["norm_title"])]
inputMovies = pd.merge(inputId, inputMovies, on="norm_title", how="inner")

# Drop unused columns if present
inputMovies = inputMovies.drop(columns=["genres", "year"], errors="ignore")

# Final tidy frame: movieId, title (no year), rating
inputMovies = inputMovies.rename(columns={"norm_title": "title"})[["movieId", "title", "rating"]]

inputMovies


,movieId,title,rating
0,34150,Fantastic Four,4.50
1,52722,Spider-Man 3,4.00
2,88140,Captain America: The First Avenger,5.00
3,95510,"Amazing Spider-Man, The",4.50
4,106072,Thor: The Dark World,4.50
5,110102,Captain America: The Winter Soldier,4.75
6,122892,Avengers: Age of Ultron,5.00
7,122902,Fantastic Four,4.50


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.


In [10]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
10246,34150,Fantastic Four (2005),"[Action, Adventure, Sci-Fi]",2005,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11838,52722,Spider-Man 3 (2007),"[Action, Adventure, Sci-Fi, Thriller, IMAX]",2007,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17544,88140,Captain America: The First Avenger (2011),"[Action, Adventure, Sci-Fi, Thriller, War]",2011,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19252,95510,"Amazing Spider-Man, The (2012)","[Action, Adventure, Sci-Fi, IMAX]",2012,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
22092,106072,Thor: The Dark World (2013),"[Action, Adventure, Fantasy, IMAX]",2013,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
23216,110102,Captain America: The Winter Soldier (2014),"[Action, Adventure, Sci-Fi, IMAX]",2014,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
26471,122892,Avengers: Age of Ultron (2015),"[Action, Adventure, Sci-Fi]",2015,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26474,122902,Fantastic Four (2015),"[Action, Adventure, Fantasy, Sci-Fi]",2015,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [11]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year',axis= 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
inputMovies['rating']

0    4.50
1    4.00
2    5.00
3    4.50
4    4.50
5    4.75
6    5.00
7    4.50
Name: rating, dtype: float64

In [13]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             36.75
Animation              0.00
Children               0.00
Comedy                 0.00
Fantasy                9.00
Romance                0.00
Drama                  0.00
Action                36.75
Crime                  0.00
Thriller               9.00
Horror                 0.00
Mystery                0.00
Sci-Fi                32.25
IMAX                  17.75
Documentary            0.00
War                    5.00
Musical                0.00
Western                0.00
Film-Noir              0.00
(no genres listed)     0.00
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.


Let's start by extracting the genre table from the original dataframe:


In [14]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
genreTable.shape

(34208, 20)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [16]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.312287
2    0.312287
3    0.000000
4    0.000000
5    0.000000
dtype: float64

In [17]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
103042    0.904437
6365      0.904437
77561     0.904437
101076    0.904437
52722     0.904437
dtype: float64

Now here's the recommendation table!


In [18]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year,norm_title
4686,4781,Megiddo: The Omega Code 2 (2001),"[Action, Adventure, Fantasy, Sci-Fi, Thriller]",2001,Megiddo: The Omega Code 2
6261,6365,"Matrix Reloaded, The (2003)","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003,"Matrix Reloaded, The"
6823,6934,"Matrix Revolutions, The (2003)","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003,"Matrix Revolutions, The"
7954,8636,Spider-Man 2 (2004),"[Action, Adventure, Sci-Fi, IMAX]",2004,Spider-Man 2
11123,46530,Superman Returns (2006),"[Action, Adventure, Sci-Fi, IMAX]",2006,Superman Returns
11838,52722,Spider-Man 3 (2007),"[Action, Adventure, Sci-Fi, Thriller, IMAX]",2007,Spider-Man 3
13705,68358,Star Trek (2009),"[Action, Adventure, Sci-Fi, IMAX]",2009,Star Trek
14397,71999,Aelita: The Queen of Mars (Aelita) (1924),"[Action, Adventure, Drama, Fantasy, Romance, S...",1924,Aelita: The Queen of Mars (Aelita)
15230,77561,Iron Man 2 (2010),"[Action, Adventure, Sci-Fi, Thriller, IMAX]",2010,Iron Man 2
16373,82461,Tron: Legacy (2010),"[Action, Adventure, Sci-Fi, IMAX]",2010,Tron: Legacy


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages

*   Learns user's preferences
*   Highly personalized for the user

##### Disadvantages

*   Doesn't take into account what others think of the item, so low quality item recommendations might happen
*   Extracting data is not always intuitive
*   Determining what characteristics of the item the user dislikes or likes is not always obvious
